In [10]:
import dash
from dash.dependencies import Output, Input
from dash import dcc, html
import plotly.graph_objs as go
from collections import deque
import websockets
import asyncio
import json
import threading
import plotly.subplots as subplots


In [21]:
WS_URL = "ws://localhost:8998/ws/real_time_data"
SENSOR_NUM = 1
sensor_data = [(deque(maxlen=100), deque(maxlen=100)) for _ in range(SENSOR_NUM)]

In [26]:
async def fetch_sensor_data():
    async with websockets.connect(WS_URL) as websocket:
        async for message in websocket:
            data = json.loads(message)
            (sensor_data[int(data["sensor_id"])][0]).append(str(data["timestamp"]))
            (sensor_data[int(data["sensor_id"])][1]).append(data["value"])

def run_fetch_data():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(fetch_sensor_data())

app = dash.Dash(__name__)
app.layout = html.Div(
    [
        dcc.Graph(id="live-graph", animate=True),
        dcc.Interval(
            id="graph-update",
            interval=1 * 1000,  # 1 second interval
            n_intervals=0,
        ),
    ]
)

@app.callback(Output("live-graph", "figure"), [Input("graph-update", "n_intervals")])
def update_graph_scatter(n):
    fig = subplots.make_subplots(rows=2, cols=2, vertical_spacing=0.2)
    for i, (X, Y) in enumerate(sensor_data):
        fig.add_trace(
            go.Scattergl(x=list(X), y=list(Y), mode="lines+markers", name=f"Sensor {i}"),
            row=(i // 2) + 1,
            col=(i % 2) + 1,
        )
        y_offset = (max(Y) - min(Y)) * 0.05
        x_offset = (max(X) - min(X)) * 0.05
        fig.update_xaxes(range=[min(X) - x_offset, max(X) + x_offset], row=(i // 2) + 1, col=(i % 2) + 1)
        fig.update_yaxes(range=[min(Y) - y_offset, max(Y) + y_offset], row=(i // 2) + 1, col=(i % 2) + 1)
    # fig.add_trace(
    #     go.Scattergl(x=list(X), y=list(Y), mode="lines+markers", name="Sensor Data")
    # )
    # y_offset = (max(Y) - min(Y)) * 0.05
    # x_offset = (max(X) - min(X)) * 0.05
    return {
        "data": fig.data,
        "layout": go.Layout(
            xaxis=dict(range=[min(X) - x_offset, max(X) + x_offset]),
            yaxis=dict(range=[min(Y) - y_offset, max(Y) + y_offset]),
        ),
    }


In [ ]:
# await (fetch_sensor_data())

In [29]:
fetch_thread = threading.Thread(target=run_fetch_data, daemon=True)
fetch_thread.start()

Exception in thread Thread-674 (run_fetch_data):
Traceback (most recent call last):
  File "c:\Users\malon\miniconda3\envs\sensors_env\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\malon\miniconda3\envs\sensors_env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\malon\miniconda3\envs\sensors_env\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\malon\AppData\Local\Temp\ipykernel_21472\3734648372.py", line 11, in run_fetch_data
  File "c:\Users\malon\miniconda3\envs\sensors_env\Lib\asyncio\base_events.py", line 687, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "C:\Users\malon\AppData\Local\Temp\ipykernel_21472\3734648372.py", line 3, in fetch_sensor_data
  File "c:\Users\malon\miniconda3\envs\sensors_env\Lib\site-packages\websockets\legacy\protocol.py", line 492, in __aiter__
    yield await self.r

In [31]:
app.run(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.